# Find the Most Representive songs of the singer

## Idea

1. Have lyrics of each singer, use that to build a VSM
2. Sort lyric vectors, find the most important terms of each song (like top 30 terms)
3. Get all the important terms of each song together, sort them by df, then choose the top [100] terms as the represent terms of this singer.
4. For each lyric vector, sum the representive terms' weight together, treat it as this song's score
5. Find the top 20 songs through the scores

In [27]:
import numpy as np
import pandas as pd
import math
import re
import pickle
from nltk.corpus import stopwords
from nltk.tokenize import regexp_tokenize
from operator import itemgetter
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Administrator\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Administrator\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [28]:
class List:
    class Node:
        def __init__(self, doc,dtf): #node는 doc_number,dtf link로 구성
            self.doc = doc
            self.dtf=dtf
            self.next = None

    def __init__(self,term): #head는 term,freq,link로 구성
        self.head = None
        self.term=term
        self.freq = 0

    def freq(self):
        return self.freq

    def term(self):
        return self.term

    def add(self,doc,dtf):
        p=self.head
        if p==None:
            self.head=self.Node(doc,dtf)
        else:
            while (p.next != None):
                p=p.next
            p.next = self.Node(doc,dtf)
        self.freq += 1

    def print_list(self): # doc_number를 출력하기 위한 함수
        p=self.head
        res=[]
        while p is not None:
            res.append(p.doc)
            p=p.next
        return res

In [29]:
def _singer_song_dic(data):
    '''Build singer-song dictionay
    
    Dictionary format:
    {singer_name: {song_name: lyrics}}
    '''
    
    singer_songs = {}
    
    for i in range(len(data)):
        singer = data.iloc[i]['artist']
        song = data.iloc[i]['song']
        lyric = data.iloc[i]['lyrics']
        
        
        if singer not in singer_songs:
            temp = {song: lyric}
            singer_songs[singer] = temp
        else:
            singer_songs[singer][song] = lyric
            
    return singer_songs

In [30]:
# This function is tested
def _update_inverted_index(name, lyrics, inverted_index):
    '''Create inverted index, count doc vector length

    Read contents form file, remove punctuation and stopwords to get terms.
    Count tf of this doc, then update inverted index.
    
    inverted_index
    '''
    
    indices = {}
    punctuation = re.compile(r'[^\w\s\']')
    
    ###
    # Count term frequency
    ###
    stop_words = set(stopwords.words('english'))
    lyrics_split = re.sub(punctuation, ' ', lyrics.lower()).split()
    
    for term in lyrics_split:
        if term in stop_words:
            continue
        elif term in indices:
            indices[term] += 1
        else:
            indices[term] = 1
    
    ###
    # Update inverted_index
    ###
    for term, frequency in indices.items():
        if term in inverted_index:
            posting = inverted_index[term]
            posting.append((name, frequency))
            inverted_index[term] = posting
        else:
            inverted_index[term] = [(name, frequency)]

In [31]:
def _compute_weight(tf, df, n_songs):
    
    idf = math.log(n_songs / df)

    tf_normalized = 1 + math.log(tf)

    weight = tf_normalized * idf
    return weight

In [32]:
def _build_song_vector(inverted_index, n_songs):
    '''Build term-weight vector for songs
    
    Compute the tf-idf weight, {term: weight}
    
    return:
        dic: a dictionary which format is {song: {term: weight}}
    '''
    
    song_vectors = {}
    
    for term, posting in inverted_index.items():
        df = len(posting)
        for pair in posting:
            song, tf = pair
            weight = _compute_weight(tf, df, n_songs)
            if song not in song_vectors:  # Create song vec
                song_vectors[song] = {term: weight}
            else:
                song_vectors[song][term] = weight  # Add new term into vec
                
    return song_vectors

In [33]:
def _get_top_songs(inverted_index, terms):
    '''Count rep terms' frequency, use that to pick rep songs
    
    Args:
        inverted_index: inverted index of this singer's songs
            format => {term: [(song, tf),]}
        terms: this singer's representive terms
    Return:
        top_songs(list): a list that contains the name of songs, order by score
    '''
    
    song_scores = {}
    for term in terms:
        posting = inverted_index[term]
        for song, _ in posting:
            if song in song_scores:
                song_scores[song] += 1
            else:
                song_scores[song] = 1
                
    top_songs = sorted(song_scores.items(), key=lambda kv: kv[1], reverse=True)
    top_songs = [name for name, _ in top_songs]
    return top_songs

In [34]:
def _find_rep_songs(songs, num_of_songs, num_of_terms):
    '''Find the most representive songs of this singer
    
    Choose songs by compare the scores which is sum the weight of each important term in that song.
    
    Args:
        songs(dic): a dictionary {song: lyrics}
        num_of_songs: number of representive songs you want to choose
        num_of_terms: number of important words we want to score the songs
        
    Return:
        rep_songs(dic): a dictionary {song: lyrics}, size will less or equal to num_of_songs
    '''
    
    inverted_index = {}
    
    n_songs = len(songs)
    if n_songs <= num_of_songs:  # Do not need to choose if not has enough songs
        return songs
    
    # Build inverted index
    for name, lyrics in songs.items():
        _update_inverted_index(name, lyrics, inverted_index)
        
    # build song vector
    song_vectors = _build_song_vector(inverted_index, n_songs)
    
    # sort by weight, get top words (put them into a set)
    selected_terms = set()
    
    for song, vector in song_vectors.items():
        sorted_v = sorted(vector.items(), key=lambda kv: kv[1], reverse=True)
        select_range = sorted_v[:num_of_terms + 1]
        for term, _ in select_range:
            selected_terms.add(term)
    
    # get score of each song
    song_sorted = _get_top_songs(inverted_index, selected_terms)
    selected_songs = song_sorted[:num_of_songs + 1]
    top_songs = {}
    for name, lyrics in songs.items():
        if name in selected_songs:
            top_songs[name] = lyrics
    
    return top_songs

In [35]:
def get_rep_songs(num_of_songs, num_of_terms, file_path):
    '''Get each singer's representive songs
    
    Args: 
        num_of_songs: number of songs you want to get from each singer
        file_path: path of input data file
    Return:
        dic: A dictionary which format is {singer, {song: lyrics}}
    '''
    
    # Read data from file
    data = pd.read_csv(file_path)
    singer_songs = _singer_song_dic(data)
    
    representive_songs = {}  # return dic
    for singer, songs in singer_songs.items():
        rep = _find_rep_songs(songs, num_of_songs, num_of_terms)
        representive_songs[singer] = rep
    
    with open('top_songs.pickle', 'wb') as handle:
        pickle.dump(representive_songs, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
        
    return representive_songs

In [36]:
# TESTing part

files = 'mylyrics00.csv'
dic=get_rep_songs(20, 50, files)

In [37]:
doc=[]
singer_list=list(dic.keys())
song_list=list(dic.values())
print((song_list[0].keys()))

stop_word = set(stopwords.words('english'))
for i in range(len(song_list)):
    for key in song_list[i].keys():
        temp=song_list[i][key]
        temp = temp.lower()
        list_temp = regexp_tokenize(temp, "[a-z]['a-z]*")

        result = []
        for w in list_temp:
            if w not in stop_word:
                result.append(w)
        
        doc.append([singer_list[i],key,result])


dict_keys(['bet-shady-2-0-cypher', 'love-game', 'rap-god', 'zane-lowe-bbc-radio-interview-part-1', 'bad-guy', 'zane-lowe-bbc-radio-interview-part-2', 'rap-god-french-version', 'evil-twin', 'westwood-freestyle-2010', 'quitter', 'just-rhymin-wit-proof', 'campaign-speech', '2-0-boys', 'loud-noises', 'shady-2-0-cypher', 'detroit-vs-everybody', 'shady-xv-cypher', 'calm-down', 'shadyxv', 'vegas', 'detroit-vs-everybody-remix'])


In [54]:
def listing(index): #LINKED LIST head:term,doc_freq // node: doc_num, freq in doc_num
    list_set=[]

    tmp_t=index[0][0]
    tmp_d=index[0][1]
    cnt=1
    for i in range(1,len(index)):
        tmp_term=index[i][0]
        tmp_doc=index[i][1]

        if(tmp_term==tmp_t and tmp_doc==tmp_d):
           cnt+=1
        else:
            list_set.append([tmp_t,tmp_d,cnt])
            cnt=1
        tmp_t=tmp_term
        tmp_d=tmp_doc
    list_set.append([tmp_t, tmp_d, cnt])

    list_all=[]
    term=list_set[0][0]
    list_t=List(term)
    list_t.add(list_set[0][1],list_set[0][2])
    for i in range(1,len(list_set)):
        if(term!=list_set[i][0]):
            list_all.append(list_t)
            list_t=List(list_set[i][0])
        list_t.add(list_set[i][1],list_set[i][2])
        term=list_set[i][0]
    list_all.append(list_t)

    return list_all

def indexing():  #vsm
    vsm_word1 = []
    index_doc=[]
    indexed_list1=[]
    for i in range(len(doc)):
        for j in range(len(doc[i][2])):
            index_doc.append([doc[i][2][j],i])

    index_doc.sort(key=itemgetter(0))

    indexed_list1=listing(index_doc)

    for i in range(len(indexed_list1)):  # 각 단어별 weight 계산 단어 1개
        vsm_word1.append([0 for j in range(len(doc) + 1)])
        vsm_word1[i][0] = indexed_list1[i].term
        p = indexed_list1[i].head
        while (p != None):
            w = (1 + math.log2(p.dtf)) * math.log2(len(doc) / indexed_list1[i].freq)
            #w=p.dtf
            vsm_word1[i][p.doc + 1] = float(w)
            p = p.next
    return vsm_word1

vsm=indexing()

In [55]:
from numpy import dot
from numpy.linalg import norm

def cos_sim(A, B):
       return dot(A, B)/(norm(A)*norm(B))

def lsa_model(str,vsm_vector):

    str=str.lower()
    temp_query=regexp_tokenize(str, "[a-z]['a-z]*")
    
    stop_word = set(stopwords.words('english'))
    list_query = []
    for w in temp_query:
        if w not in stop_word:
            list_query.append(w)

    vsm_=np.zeros((len(vsm_vector),len(vsm_vector[0])-1))
    query=np.zeros((len(vsm_vector),1))

    for i in range(len(vsm_)): #vsm
        for j in range(len(vsm_[i])):
            vsm_[i][j]=vsm_vector[i][j+1]

        for a in range(len(list_query)):
            if vsm_vector[i][0] == list_query[a]:
                query[i][0] = query[i][0] + 1
                
    print("Vector space model(word by num_doc) :")
    print(vsm_)
    print("Query(word by num_doc) :")
    print(query)
    
    sim=np.zeros(len(vsm_[0]))
    
#     vsm_t=vsm_.T
#     query_t=query.T
    
    
#     for i in range(len(vsm_t)):
#         sim[i]=cos_sim(vsm_t[i],query_t[0])
        
        
    U, S, V_T = np.linalg.svd(vsm_, full_matrices=False) #SVD 적용 해서 vsm을 u,s,v_T 로 분해해

    print("matrix U(num_doc by num_doc) : ")
    print(U)
    print("maxtrix S : ")
    print(S)
    print("matrix V_T(num_doc by num_doc) : ")
    print(V_T)

    k = 2

    U_k=np.zeros((len(U),k))
    S_k=np.zeros((k,k))
    V_T_k=np.zeros((k,len(vsm_[0])))


    for i in range(len(U)):   # k 값에 대해 dimension 축소
        for j in range(0,k):
            U_k[i][j]=U[i][j]
    for i in range(0,k):
        S_k[i][i]=S[i]
    for i in range(0,k):
        for j in range(0,len(vsm_[0])):
            V_T_k[i][j]=V_T[i][j]

    S_k_inv=np.linalg.inv(S_k)
    q=np.dot(query.T,np.dot(U_k,S_k_inv))
    
    V_k=V_T_k.T

    
    print(q[0])
    print(V_k[0])
    
    for i in range((len(V_k))): #cos similarity 계산
        sim[i]=cos_sim(V_k[i],q[0])
    
    
    result=[]
    song_top_num=len(song_list[0])

    index_art=0
    sum=0
    
    for i in range(len(sim)):
        
        if i%song_top_num==0 and i>=1:
            result.append([singer_list[index_art],sum/song_top_num])
            sum=0
            index_art+=1
        
        sum+=sim[i]

    
    result.append([singer_list[index_art],sum/song_top_num])
    result.sort(key=itemgetter(1),reverse=True)    
    

    print("결과값 :")
    for i in range(len(result)):
        print("artist : ",result[i][0],"// similarity : ",result[i][1])



In [56]:
sample_data=pd.read_csv(files)

lsa_model(sample_data.iloc[0]['lyrics'],vsm)

Vector space model(word by num_doc) :
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
Query(word by num_doc) :
[[0.]
 [0.]
 [0.]
 ...
 [0.]
 [0.]
 [0.]]
matrix U(num_doc by num_doc) : 
[[-7.03665342e-04 -4.48877848e-04  5.96292718e-04 ...  1.28075409e-02
   1.67776987e-02  4.30655221e-03]
 [-3.33580995e-04  6.44764244e-05  2.03441328e-04 ... -3.33190724e-03
  -3.88507095e-04  1.52850319e-02]
 [-3.17777313e-04 -2.94776931e-04  3.65318737e-04 ...  1.45789960e-02
   3.55395216e-02 -4.32629073e-04]
 ...
 [-1.55887532e-03  1.28260897e-03  3.57972385e-04 ... -9.06150761e-04
   6.94025501e-04 -6.82541145e-05]
 [-1.17217924e-03 -5.39797755e-04  5.01496487e-04 ...  5.24864579e-04
  -8.85326904e-04 -9.46368933e-04]
 [-1.40482090e-03 -8.31144787e-04  2.67664114e-04 ... -2.35618137e-03
  -3.63470899e-04  4.12784947e-04]]
maxtrix S : 
[4.26135456e+02 2.62428782e+02 2.49349706e+02 2.18196854e+